In [2]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import os

In [4]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

In [5]:
x_train = x_train/255
x_test = x_test/255
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [7]:
#ResNet Block
def resent_block(inputs,num_filters=16,
                kernel_size=3,strides=1,
                activation='relu'):
    x = Conv2D(num_filters,kernel_size,strides,padding='same',
              kernel_initializer='he_normal',kernel_regularize=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    if(activation):
        x = Activation('relu')(x)
    return x

In [1]:
#建一个20层的ResNet网络
def resnet_v1(input_shape):
    inputs = Input(shape=input_shape)
    
    #第一层
    x = resnet_block(inputs)
    print('layer1,xshape:',x.shape)
    #第二到七层
    for i in range(6):
        a = resent_block(inputs=x)
        b = resent_block(inputs=a,activation=None)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    #out:32*32*16
    #第8-13层
    for i in range(6):
        if i==0:
            a = resent_block(inputs=x,strides=2,num_filters=32)
        else:
            a = resent_block(inputs=x,num_filters=32)
        b = resent_block(inputs=a,activation=None,num_filters=32)
        if i==0:
            x = Conv2D(32,kernel_size=3,strides=2,padding='same',
                      kernel_initializer='he_normal',kernel_regularizer=l2(1e-2))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    #out:16*16*32
    #第14-19层
    for i in range(6):
        if i==0:
            a = resnet_block(inputs = x,strides=2,num_filters=64)
        else:
            a = resnet_block(inputs = x,num_filters=64)
            
        b = resnet_block(inputs=a,activation=None,num_filters=64)
        if i==0:
            x = Conv2D(64,kernel_size=3,strides=2,padding='same',
                      kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')
    out:8*8*64
    #第20层
    x = AveragePooling2D(pool_size=2)(x)
    #out:4*4*64
    y = Flatten()(x)
    #out=1024
    outputs = Dense(10,activation='softmax',
                   kernel_initializer='he_normal')(x)
    #初始化模型
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [3]:
model = resnet_v1((32,32,3))

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy'])

model.summary()

NameError: name 'Input' is not defined

In [ ]:
checkpoint = ModelCheckpoint(filepath='./cifar10_resent_ckpt.h5',monitor='val_acc',
                            verbose=1,save_best_omly=True)

def lr_sch(epoch):
    #200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5
lr_scheduler = LearningRateScheduler(lr_sch)
lr_reducer = ReduceLROnPlateau(monitor='val_acc',factor=0.2,patience=5,
                              mode='max',min_lr=1e-3)
callbacks = [checkpoint,lr_schedulr,lr_reducer]

In [ ]:
model.fit(x_train,y_train,batch_size=64,epochs=200,validation_data=(x_test,y_test),verbose=1,callbacks=callbacks)

In [ ]:
scores = model.evaluate(x_test,y_test,verbose=1)
print('test loss:',scores[0])
print('test accuracy:',scores[1])